# 6. Modelos de Regressão


Este notebook explora diferentes modelos de regressão para análise de dados, incluindo Regressão Linear e Lasso, utilizando técnicas de pré-processamento como transformação Box-Cox e seleção de variáveis. O objetivo é avaliar o desempenho dos modelos em termos de métricas como RMSE e R², tanto nos datasets completos quanto após a remoção de outliers das questões do Enem (vetorizadas).


In [1]:
# Importando Dependências para Modelos de Regressão
import pandas as pd
import numpy as np
from scipy import stats
import statsmodels.api as sm

from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.linear_model import LassoCV

In [2]:
# Leitura dos dados
enem_data = pd.read_pickle("../data/final/enem_data_final.pkl")
enem_data.head()

,numero_questao,gabarito,NU_PARAM_A,nu_param_B,NU_PARAM_C,ANO,enunciado,alternativas,gabarito_texto,distratores,...,enunciado_embbedings_word2vec_50,gabarito_embbedings_word2vec_50,distratores_embbedings_word2vec_50,similaridade_enunciado_gabarito_50,similaridade_enunciado_distratores_50,similaridade_gabarito_distratores_50,resposta_llama,acerto_llamma,resposta_deepseek,acerto_deepseek
0,1,C,3.43894,0.97831,0.10855,2017,"No império africano do Mali, no século XIV, To...",A: isolamento geográﬁco do Saara ocidental; B...,posição relativa nas redes de circulação,isolamento geográﬁco do Saara ocidental; explo...,...,"[-0.017395772, -0.004402813, -0.046908338, 0.0...","[-0.16301225, -0.142375, 0.012171499, -0.00457...","[-0.041815996, -0.06492707, 0.0900044, -0.0498...",0.141353,0.516898,0.151251,C,True,C,True
1,2,D,3.00837,0.49169,0.13877,2017,Após a Declaração Universal dos Direitos Human...,A: ataque feito pelos japoneses à base milita...,execução de judeus e eslavos presos em guetos ...,ataque feito pelos japoneses à base militar am...,...,"[-0.051949076, 0.03128451, -0.072241075, 0.028...","[-0.18672037, 0.04755275, -0.17406088, -0.0398...","[0.05533243, -0.039066706, -0.059438962, 0.019...",0.422576,0.313321,0.423611,D,True,D,True
2,3,D,0.60432,3.25992,0.08798,2017,"A moralidade, Bentham exortava, não é uma ques...",A: fundamentação cientíﬁca de viés positivist...,racionalidade de caráter pragmático,fundamentação cientíﬁca de viés positivista; c...,...,"[0.026049094, 0.043251127, -0.029757027, -0.00...","[0.011740998, 0.07592634, -0.016144669, 0.1384...","[0.06667993, 0.015803386, 0.029443847, 0.11312...",0.427063,0.471643,0.602545,D,True,D,True
3,4,E,1.85031,0.57925,0.11344,2017,Fala-se muito nos dias de hoje em direitos do ...,A: modernização da educação escolar; B: atuali...,universalização do princípio da igualdade civil,modernização da educação escolar; atualização ...,...,"[-0.005251272, 0.039871164, -0.018038195, 0.06...","[-0.073669255, 0.12609875, 0.063628, 0.1261957...","[-0.07921684, 0.10788742, 0.10466367, -0.04121...",0.320939,0.482462,0.421501,E,True,E,True
4,5,C,2.46290,0.76307,0.17672,2017,Na Constituição da República Federativa do Bra...,A: etnia e miscigenação racial; B: sociedade...,espaço e sobrevivência cultural,etnia e miscigenação racial; sociedade e igual...,...,"[0.01738678, 0.0033733728, -0.027247325, 0.002...","[0.11709199, -0.16062833, 0.082869664, -0.1195...","[-0.02205533, 0.08129784, 0.10641242, -0.05484...",0.304050,0.501346,0.411802,C,True,C,True


In [3]:
enem_data = enem_data[enem_data["nu_param_B"].notna()]

---

## 6.1. Embeddings de 300 Dimensões


In [4]:
# Coletando os dados
X_300 = [
    np.array(embedding) for embedding in enem_data["enunciado_embbedings_word2vec_300"]
]  # Somente Embeddings 300

y_300 = enem_data["nu_param_B"]  # Parâmetro de Dificuldade (B)

In [5]:
# Aplicando Transformações
add_list = [(y_300.min() * (-1)) + 1] * len(y_300)
y_300 = y_300 + add_list

# Aplicando Box-Cox
y_300, best_lambda = stats.boxcox(y_300)
print(best_lambda)

0.6304389864783265


### 6.1.1. Regressão Linear


In [6]:
# Separando em conjunto de treino e teste
X_train_300, X_test_300, y_train_300, y_test_300 = train_test_split(
    X_300, y_300, test_size=0.3, random_state=42
)

In [7]:
# Adicionando constante
X_train_300 = sm.add_constant(X_train_300)
X_test_300 = sm.add_constant(X_test_300)

In [8]:
# Criando o modelo e realizando predição
linear_model_300 = sm.OLS(y_train_300, X_train_300).fit()
prediction_linear_model_300 = linear_model_300.predict(X_test_300)

In [9]:
# Cálculo do RMSE
rmse_300 = root_mean_squared_error(y_test_300, prediction_linear_model_300)
print("RMSE: ", rmse_300)

RMSE:  0.7720791146717556


In [10]:
# Visualização do modelo e resultados
print(linear_model_300.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 10 Jun 2025   Prob (F-statistic):                nan
Time:                        09:27:19   Log-Likelihood:                 5959.2
No. Observations:                 186   AIC:                        -1.155e+04
Df Residuals:                       0   BIC:                        -1.095e+04
Df Model:                         185                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2107        inf          0        n

/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


### 6.1.2. Regressão Lasso


In [11]:
# Separando em conjunto de treino e teste
X_train_300_lasso, X_test_300_lasso, y_train_300_lasso, y_test_300_lasso = (
    train_test_split(X_300, y_300, test_size=0.3, random_state=42)
)

In [12]:
lasso_model_300 = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0).fit(
    X_train_300_lasso, y_train_300_lasso
)
pred_lasso_model_300 = lasso_model_300.predict(X_test_300_lasso)

print("Melhor R2 Score:", lasso_model_300.score(X_train_300_lasso, y_train_300_lasso))
print("Melhor Alpha:", lasso_model_300.alpha_)
print(
    "RMSE com o alpha escolhido:",
    root_mean_squared_error(y_test_300_lasso, pred_lasso_model_300),
)

Melhor R2 Score: 0.5344153222227536
Melhor Alpha: 0.001
RMSE com o alpha escolhido: 0.5177163330655292


### 6.1.3. Regressões com Corte (-3,3)


In [13]:
# Aplicando corte de dificuldade sugerido pelo artigo motivador (-3, 3)
enem_filtered = enem_data.copy()

enem_filtered = enem_filtered[
    (enem_filtered["nu_param_B"] >= -3) & (enem_filtered["nu_param_B"] <= 3)
]
enem_filtered["nu_param_B"].describe()

count    263.000000
mean       1.075595
std        0.661348
min       -0.726450
25%        0.612925
50%        1.051180
75%        1.535350
max        2.802770
Name: nu_param_B, dtype: float64

In [14]:
# Coletando os dados
X_300_filtered = [
    np.array(embedding)
    for embedding in enem_filtered["enunciado_embbedings_word2vec_300"]
]
y_300_filtered = enem_filtered["nu_param_B"]

In [15]:
# Aplicando Transformações
add_list = [(y_300_filtered.min() * (-1)) + 1] * len(y_300_filtered)
y_300_filtered = y_300_filtered + add_list

# Aplicando Boxcox
y_300_filtered, best_lambda = stats.boxcox(y_300_filtered)
print(best_lambda)

0.9332701631134968


### 6.1.3.A) Regressão Linear


In [16]:
# Dividindo os dados em treino e teste
X_train_300_filtered, X_test_300_filtered, y_train_300_filtered, y_test_300_filtered = (
    train_test_split(X_300_filtered, y_300_filtered, test_size=0.3, random_state=42)
)

In [17]:
# Adicionando constante
X_train_300_filtered = sm.add_constant(X_train_300_filtered)
X_test_300_filtered = sm.add_constant(X_test_300_filtered)

In [18]:
# Criando o modelo e realizando a predição
linear_model_300_filtered = sm.OLS(y_train_300_filtered, X_train_300_filtered).fit()
pred_linear_model_300_filtered = linear_model_300_filtered.predict(X_test_300_filtered)

In [19]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_300_filtered, pred_linear_model_300_filtered)
print("RMSE:", rms)

RMSE: 1.2188517275533697


In [20]:
print(linear_model_300_filtered.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 10 Jun 2025   Prob (F-statistic):                nan
Time:                        09:27:20   Log-Likelihood:                 5800.6
No. Observations:                 184   AIC:                        -1.123e+04
Df Residuals:                       0   BIC:                        -1.064e+04
Df Model:                         183                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4067        inf          0        n

/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


### 6.1.3.B) Regressão Lasso


In [21]:
# Dividindo os dados em treino e teste
(
    X_train_300_filtered_lasso,
    X_test_300_filtered_lasso,
    y_train_300_filtered_lasso,
    y_test_300_filtered_lasso,
) = train_test_split(X_300_filtered, y_300_filtered, test_size=0.3, random_state=42)

In [22]:
lasso_model_300_filtered = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100], random_state=0, tol=0.1
).fit(X_train_300_filtered_lasso, y_train_300_filtered_lasso)

pred_300_lasso_filtered = lasso_model_300_filtered.predict(X_test_300_filtered_lasso)

print(
    "Melhor R2 score:",
    lasso_model_300_filtered.score(
        X_train_300_filtered_lasso, y_train_300_filtered_lasso
    ),
)
print("Melhor Alpha:", lasso_model_300_filtered.alpha_)
print(
    "RMSE com o Alpha escolhido:",
    root_mean_squared_error(y_test_300_filtered_lasso, pred_300_lasso_filtered),
)

Melhor R2 score: 0.5662683125013057
Melhor Alpha: 0.001
RMSE com o Alpha escolhido: 0.6342396428478064


### 6.1.4. LLM & Similaridade Features


In [23]:
# Coletando os dados
y_300_all_features = enem_data["nu_param_B"]
X_300_all_features = [
    np.concatenate(
        [
            np.array(embedding),
            [
                acerto_deepseek,
                acerto_llamma,
                similaridade_enunciado_gabarito_300,
                similaridade_enunciado_distratores_300,
                similaridade_gabarito_distratores_300,
            ],
        ]
    )
    for acerto_deepseek, acerto_llamma, embedding, similaridade_enunciado_gabarito_300, similaridade_enunciado_distratores_300, similaridade_gabarito_distratores_300 in zip(
        enem_data["acerto_deepseek"],
        enem_data["acerto_llamma"],
        enem_data["enunciado_embbedings_word2vec_300"],
        enem_data["similaridade_enunciado_gabarito_300"],
        enem_data["similaridade_enunciado_distratores_300"],
        enem_data["similaridade_gabarito_distratores_300"],
    )
]

In [24]:
# Aplicando Transformações
add_list = [(y_300_all_features.min() * (-1)) + 1] * len(y_300_all_features)
y_300_all_features = y_300_all_features + add_list

# Aplicando Boxcox
y_300_all_features, best_lambda = stats.boxcox(y_300_all_features)
print(best_lambda)

0.6304389864783265


### 6.1.4.A) Regressão linear


In [25]:
# Separando em conjunto de treino e teste

(
    X_train_300_all_features,
    X_test_300_all_features,
    y_train_300_all_features,
    y_test_300_all_features,
) = train_test_split(
    X_300_all_features, y_300_all_features, test_size=0.3, random_state=42
)

In [26]:
# Adicionando constante
X_train_300_all_features = sm.add_constant(X_train_300_all_features)
X_test_300_all_features = sm.add_constant(X_test_300_all_features)

In [27]:
# Criando o modelo e realizando predição
linear_model_300_all_features = sm.OLS(
    y_train_300_all_features, X_train_300_all_features
).fit()
pred_linear_model_300_all_features = linear_model_300_all_features.predict(
    X_test_300_all_features
)

In [28]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_300_all_features, pred_linear_model_300_all_features)
print("RMSE", rms)

RMSE 0.7478685280228783


In [29]:
print(linear_model_300_all_features.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 10 Jun 2025   Prob (F-statistic):                nan
Time:                        09:27:20   Log-Likelihood:                 5853.4
No. Observations:                 186   AIC:                        -1.133e+04
Df Residuals:                       0   BIC:                        -1.073e+04
Df Model:                         185                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0449        inf          0        n

/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


### 6.1.4.B) Lasso


In [30]:
# Separando em conjunto de treino e teste
(
    X_train_300_lasso_all_features,
    X_test_300_lasso_all_features,
    y_train_300_lasso_all_features,
    y_test_300_lasso_all_features,
) = train_test_split(
    X_300_all_features, y_300_all_features, test_size=0.3, random_state=42
)

In [31]:
lasso_model_all_features_300 = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0
).fit(X_train_300_lasso_all_features, y_train_300_lasso_all_features)

pred_300_lasso_all_features = lasso_model_all_features_300.predict(
    X_test_300_lasso_all_features
)

print(
    "melhor r2 score:",
    lasso_model_all_features_300.score(
        X_train_300_lasso_all_features, y_train_300_lasso_all_features
    ),
)
print("melhor alpha:", lasso_model_all_features_300.alpha_)
print(
    "RMSE com o alpha escolhido:",
    root_mean_squared_error(y_test_300_lasso_all_features, pred_300_lasso_all_features),
)

melhor r2 score: 0.547574748756191
melhor alpha: 0.001
RMSE com o alpha escolhido: 0.48813975574107166


---

## 6.2. Embeddings de 100 Dimensões


In [32]:
# Coletando os dados
X_100 = [
    np.array(embedding) for embedding in enem_data["enunciado_embbedings_word2vec_100"]
]  # Somente Embeddings

y_100 = enem_data["nu_param_B"]  # Parâmetro de Dificuldade (B)

In [33]:
# Aplicando Transformações
add_list = [(y_100.min() * (-1)) + 1] * len(y_100)
y_100 = y_100 + add_list

# Aplicando Box-Cox
y_100, best_lambda = stats.boxcox(y_100)
print(best_lambda)

0.6304389864783265


### 6.2.1. Regressão Linear


In [34]:
# Separando em conjunto de treino e teste
X_train_100, X_test_100, y_train_100, y_test_100 = train_test_split(
    X_100, y_100, test_size=0.3, random_state=42
)

In [35]:
# Adicionando constante
X_train_100 = sm.add_constant(X_train_100)
X_test_100 = sm.add_constant(X_test_100)

In [36]:
# Criando o modelo e realizando predição
linear_model_100 = sm.OLS(y_train_100, X_train_100).fit()
pred_linear_model_100 = linear_model_100.predict(X_test_100)

In [37]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_100, pred_linear_model_100)
print("RMSE: ", rms)

RMSE:  0.7720791146717556


In [38]:
# Visualização do modelo e resultados
print(linear_model_100.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 10 Jun 2025   Prob (F-statistic):                nan
Time:                        09:27:20   Log-Likelihood:                 5959.2
No. Observations:                 186   AIC:                        -1.155e+04
Df Residuals:                       0   BIC:                        -1.095e+04
Df Model:                         185                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2107        inf          0        n

/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


### 6.2.2. Regressão Lasso


In [39]:
# Separando em conjunto de treino e teste
X_train_100_lasso, X_test_100_lasso, y_train_100_lasso, y_test_100_lasso = (
    train_test_split(X_100, y_100, test_size=0.3, random_state=42)
)

In [40]:
model_lasso = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0).fit(
    X_train_100_lasso, y_train_100_lasso
)
pred_lasso = model_lasso.predict(X_test_100_lasso)

print("Melhor R2 Score:", model_lasso.score(X_train_100_lasso, y_train_100_lasso))
print("Melhor Alpha:", model_lasso.alpha_)
print(
    "RMSE com o alpha escolhido:", root_mean_squared_error(y_test_100_lasso, pred_lasso)
)

Melhor R2 Score: 0.5344153222227536
Melhor Alpha: 0.001
RMSE com o alpha escolhido: 0.5177163330655292


### 6.2.3. Regressões com Corte (-3,3)


In [41]:
# Aplicando corte de dificuldade sugerido pelo artigo motivador (-3, 3)
enem_filtered = enem_data.copy()

enem_filtered = enem_filtered[
    (enem_filtered["nu_param_B"] >= -3) & (enem_filtered["nu_param_B"] <= 3)
]
enem_filtered["nu_param_B"].describe()

count    263.000000
mean       1.075595
std        0.661348
min       -0.726450
25%        0.612925
50%        1.051180
75%        1.535350
max        2.802770
Name: nu_param_B, dtype: float64

In [42]:
# Coletando os dados
X_100_filtered = [
    np.array(embedding)
    for embedding in enem_filtered["enunciado_embbedings_word2vec_100"]
]
y_100_filtered = enem_filtered["nu_param_B"]

In [43]:
# Aplicando Transformações
add_list = [(y_100_filtered.min() * (-1)) + 1] * len(y_100_filtered)
y_100_filtered = y_100_filtered + add_list

# Aplicando Boxcox
y_100_filtered, best_lambda = stats.boxcox(y_100_filtered)
print(best_lambda)

0.9332701631134968


### 6.2.3.A) Regressão Linear


In [44]:
# Dividindo os dados em treino e teste
X_train_100_filtered, X_test_100_filtered, y_train_100_filtered, y_test_100_filtered = (
    train_test_split(X_100_filtered, y_100_filtered, test_size=0.3, random_state=42)
)

In [45]:
# Adicionando constante
X_train_100_filtered = sm.add_constant(X_train_100_filtered)
X_test_100_filtered = sm.add_constant(X_test_100_filtered)

In [46]:
# Criando o modelo e realizando a predição
linear_model_100_filtered = sm.OLS(y_train_100_filtered, X_train_100_filtered).fit()
pred_linear_model_100_filtered = linear_model_100_filtered.predict(X_test_100_filtered)

In [47]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_100_filtered, pred_linear_model_100_filtered)
print("RMSE:", rms)

RMSE: 1.2188517275533697


In [48]:
print(linear_model_100_filtered.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 10 Jun 2025   Prob (F-statistic):                nan
Time:                        09:27:21   Log-Likelihood:                 5800.6
No. Observations:                 184   AIC:                        -1.123e+04
Df Residuals:                       0   BIC:                        -1.064e+04
Df Model:                         183                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4067        inf          0        n

/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


### 6.2.3.B) Regressão Lasso


In [49]:
# Dividindo os dados em treino e teste
(
    X_train_100_filtered_lasso,
    X_test_100_filtered_lasso,
    y_train_100_filtered_lasso,
    y_test_100_filtered_lasso,
) = train_test_split(X_100_filtered, y_100_filtered, test_size=0.3, random_state=42)

In [50]:
lasso_model_100_filtered = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100], random_state=0, tol=0.1
).fit(X_train_100_filtered_lasso, y_train_100_filtered_lasso)

pred_lasso_filtered_100 = lasso_model_100_filtered.predict(X_test_100_filtered_lasso)

print(
    "Melhor R2 score:",
    lasso_model_100_filtered.score(
        X_train_100_filtered_lasso, y_train_100_filtered_lasso
    ),
)
print("Melhor Alpha:", lasso_model_100_filtered.alpha_)
print(
    "RMSE com o Alpha escolhido:",
    root_mean_squared_error(y_test_100_filtered_lasso, pred_lasso_filtered_100),
)

Melhor R2 score: 0.5662683125013057
Melhor Alpha: 0.001
RMSE com o Alpha escolhido: 0.6342396428478064


### 6.2.4. LLM & Similaridade Features


In [51]:
# Coletando os dados
y_100_all_features = enem_data["nu_param_B"]
X_100_all_features = [
    np.concatenate(
        [
            np.array(embedding),
            [
                acerto_deepseek,
                acerto_llamma,
                similaridade_enunciado_gabarito_100,
                similaridade_enunciado_distratores_100,
                similaridade_gabarito_distratores_100,
            ],
        ]
    )
    for acerto_deepseek, acerto_llamma, embedding, similaridade_enunciado_gabarito_100, similaridade_enunciado_distratores_100, similaridade_gabarito_distratores_100 in zip(
        enem_data["acerto_deepseek"],
        enem_data["acerto_llamma"],
        enem_data["enunciado_embbedings_word2vec_100"],
        enem_data["similaridade_enunciado_gabarito_100"],
        enem_data["similaridade_enunciado_distratores_100"],
        enem_data["similaridade_gabarito_distratores_100"],
    )
]

In [52]:
# Aplicando Transformações
add_list = [(y_100_all_features.min() * (-1)) + 1] * len(y_100_all_features)
y_100_all_features = y_100_all_features + add_list

# Aplicando Boxcox
y_100_all_features, best_lambda = stats.boxcox(y_100_all_features)
print(best_lambda)

0.6304389864783265


### 6.2.4.A) Regressão linear


In [53]:
# Separando em conjunto de treino e teste

(
    X_train_100_all_features,
    X_test_100_all_features,
    y_train_100_all_features,
    y_test_100_all_features,
) = train_test_split(
    X_100_all_features, y_100_all_features, test_size=0.3, random_state=42
)

In [54]:
# Adicionando constante
X_train_100_all_features = sm.add_constant(X_train_100_all_features)
X_test_100_all_features = sm.add_constant(X_test_100_all_features)

In [55]:
# Criando o modelo e realizando predição
linear_model_100_all_features = sm.OLS(
    y_train_100_all_features, X_train_100_all_features
).fit()
pred_linear_model_100_all_features = linear_model_100_all_features.predict(
    X_test_100_all_features
)

In [56]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_100_all_features, pred_linear_model_100_all_features)
print("RMSE", rms)

RMSE 0.7478685280228783


In [57]:
print(linear_model_100_all_features.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 10 Jun 2025   Prob (F-statistic):                nan
Time:                        09:27:21   Log-Likelihood:                 5853.4
No. Observations:                 186   AIC:                        -1.133e+04
Df Residuals:                       0   BIC:                        -1.073e+04
Df Model:                         185                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0449        inf          0        n

/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


### 6.2.4.B) Lasso


In [58]:
# Separando em conjunto de treino e teste
(
    X_train_100_lasso_all_features,
    X_test_100_lasso_all_features,
    y_train_100_lasso_all_features,
    y_test_100_lasso_all_features,
) = train_test_split(
    X_100_all_features, y_100_all_features, test_size=0.3, random_state=42
)

In [59]:
lasso_model_all_features_100 = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0
).fit(X_train_100_lasso_all_features, y_train_100_lasso_all_features)

pred_100_lasso_all_features = lasso_model_all_features_100.predict(
    X_test_100_lasso_all_features
)

print(
    "melhor r2 score:",
    lasso_model_all_features_100.score(
        X_train_100_lasso_all_features, y_train_100_lasso_all_features
    ),
)
print("melhor alpha:", lasso_model_all_features_100.alpha_)
print(
    "RMSE com o alpha escolhido:",
    root_mean_squared_error(y_test_100_lasso_all_features, pred_100_lasso_all_features),
)

melhor r2 score: 0.547574748756191
melhor alpha: 0.001
RMSE com o alpha escolhido: 0.48813975574107166


---

## 6.3. Embeddings de 50 Dimensões


In [60]:
# Coletando os dados
X_50 = [
    np.array(embedding) for embedding in enem_data["enunciado_embbedings_word2vec_50"]
]  # Somente Embeddings 50

y_50 = enem_data["nu_param_B"]  # Parâmetro de Dificuldade (B)

In [61]:
# Aplicando Transformações
add_list = [(y_50.min() * (-1)) + 1] * len(y_50)
y_50 = y_50 + add_list

# Aplicando Box-Cox
y_50, best_lambda = stats.boxcox(y_50)
print(best_lambda)

0.6304389864783265


### 6.3.1. Regressão Linear


In [62]:
# Separando em conjunto de treino e teste
X_train_50, X_test_50, y_train_50, y_test_50 = train_test_split(
    X_50, y_50, test_size=0.3, random_state=42
)

In [63]:
# Adicionando constante
X_train_50 = sm.add_constant(X_train_50)
X_test_50 = sm.add_constant(X_test_50)

In [64]:
# Criando o modelo e realizando predição
linear_model_50 = sm.OLS(y_train_50, X_train_50).fit()
pred_linear_model_50 = linear_model_50.predict(X_test_50)

In [65]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_50, pred_linear_model_50)
print("RMSE: ", rms)

RMSE:  0.7720791146717556


In [66]:
# Visualização do modelo e resultados
print(linear_model_50.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 10 Jun 2025   Prob (F-statistic):                nan
Time:                        09:27:21   Log-Likelihood:                 5959.2
No. Observations:                 186   AIC:                        -1.155e+04
Df Residuals:                       0   BIC:                        -1.095e+04
Df Model:                         185                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.2107        inf          0        n

/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


### 6.3.2. Regressão Lasso


In [67]:
# Separando em conjunto de treino e teste
X_train_50_lasso, X_test_50_lasso, y_train_50_lasso, y_test_50_lasso = train_test_split(
    X_50, y_50, test_size=0.3, random_state=42
)

In [68]:
lasso_model_50 = LassoCV(alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0).fit(
    X_train_50_lasso, y_train_50_lasso
)
pred_lasso_model_50 = lasso_model_50.predict(X_test_50_lasso)

print("Melhor R2 Score:", lasso_model_50.score(X_train_50_lasso, y_train_50_lasso))
print("Melhor Alpha:", lasso_model_50.alpha_)
print(
    "RMSE com o alpha escolhido:",
    root_mean_squared_error(y_test_50_lasso, pred_lasso_model_50),
)

Melhor R2 Score: 0.5344153222227536
Melhor Alpha: 0.001
RMSE com o alpha escolhido: 0.5177163330655292


### 6.3.3. Regressões com Corte (-3,3)


In [69]:
# Aplicando corte de dificuldade sugerido pelo artigo motivador (-3, 3)
enem_filtered = enem_data.copy()

enem_filtered = enem_filtered[
    (enem_filtered["nu_param_B"] >= -3) & (enem_filtered["nu_param_B"] <= 3)
]
enem_filtered["nu_param_B"].describe()

count    263.000000
mean       1.075595
std        0.661348
min       -0.726450
25%        0.612925
50%        1.051180
75%        1.535350
max        2.802770
Name: nu_param_B, dtype: float64

In [70]:
# Coletando os dados
X_50_filtered = [
    np.array(embedding)
    for embedding in enem_filtered["enunciado_embbedings_word2vec_50"]
]
y_50_filtered = enem_filtered["nu_param_B"]

In [71]:
# Aplicando Transformações
add_list = [(y_50_filtered.min() * (-1)) + 1] * len(y_50_filtered)
y_50_filtered = y_50_filtered + add_list

# Aplicando Boxcox
y_50_filtered, best_lambda = stats.boxcox(y_50_filtered)
print(best_lambda)

0.9332701631134968


### 6.3.3.A) Regressão Linear


In [72]:
# Dividindo os dados em treino e teste
X_train_50_filtered, X_test_50_filtered, y_train_50_filtered, y_test_50_filtered = (
    train_test_split(X_50_filtered, y_50_filtered, test_size=0.3, random_state=42)
)

In [73]:
# Adicionando constante
X_train_50_filtered = sm.add_constant(X_train_50_filtered)
X_test_50_filtered = sm.add_constant(X_test_50_filtered)

In [74]:
# Criando o modelo e realizando a predição
linear_model_50_filtered = sm.OLS(y_train_50_filtered, X_train_50_filtered).fit()
pred_linear_model_50_filtered = linear_model_50_filtered.predict(X_test_50_filtered)

In [75]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_50_filtered, pred_linear_model_50_filtered)
print("RMSE:", rms)

RMSE: 1.2188517275533697


In [76]:
print(linear_model_50_filtered.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 10 Jun 2025   Prob (F-statistic):                nan
Time:                        09:27:22   Log-Likelihood:                 5800.6
No. Observations:                 184   AIC:                        -1.123e+04
Df Residuals:                       0   BIC:                        -1.064e+04
Df Model:                         183                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          2.4067        inf          0        n

/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


### 6.3.3.B) Regressão Lasso


In [77]:
# Dividindo os dados em treino e teste
(
    X_train_50_filtered_lasso,
    X_test_50_filtered_lasso,
    y_train_50_filtered_lasso,
    y_test_50_filtered_lasso,
) = train_test_split(X_50_filtered, y_50_filtered, test_size=0.3, random_state=42)

In [78]:
lasso_model_50_filtered = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10, 100], random_state=0, tol=0.1
).fit(X_train_50_filtered_lasso, y_train_50_filtered_lasso)

pred_lasso_model_50_filtered = lasso_model_50_filtered.predict(X_test_50_filtered_lasso)

print(
    "Melhor R2 score:",
    lasso_model_50_filtered.score(X_train_50_filtered_lasso, y_train_50_filtered_lasso),
)
print("Melhor Alpha:", lasso_model_50_filtered.alpha_)
print(
    "RMSE com o Alpha escolhido:",
    root_mean_squared_error(y_test_50_filtered_lasso, pred_lasso_model_50_filtered),
)

Melhor R2 score: 0.5662683125013057
Melhor Alpha: 0.001
RMSE com o Alpha escolhido: 0.6342396428478064


### 6.3.4. LLM & Similaridade Features


In [79]:
# Coletando os dados
y_50_all_features = enem_data["nu_param_B"]
X_50_all_features = [
    np.concatenate(
        [
            np.array(embedding),
            [
                acerto_deepseek,
                acerto_llamma,
                similaridade_enunciado_gabarito_50,
                similaridade_enunciado_distratores_50,
                similaridade_gabarito_distratores_50,
            ],
        ]
    )
    for acerto_deepseek, acerto_llamma, embedding, similaridade_enunciado_gabarito_50, similaridade_enunciado_distratores_50, similaridade_gabarito_distratores_50 in zip(
        enem_data["acerto_deepseek"],
        enem_data["acerto_llamma"],
        enem_data["enunciado_embbedings_word2vec_50"],
        enem_data["similaridade_enunciado_gabarito_50"],
        enem_data["similaridade_enunciado_distratores_50"],
        enem_data["similaridade_gabarito_distratores_50"],
    )
]

In [80]:
# Aplicando Transformações
add_list = [(y_50_all_features.min() * (-1)) + 1] * len(y_50_all_features)
y_50_all_features = y_50_all_features + add_list

# Aplicando Boxcox
y_50_all_features, best_lambda = stats.boxcox(y_50_all_features)
print(best_lambda)

0.6304389864783265


### 6.3.4.A) Regressão linear


In [81]:
# Separando em conjunto de treino e teste

(
    X_train_50_all_features,
    X_test_50_all_features,
    y_train_50_all_features,
    y_test_50_all_features,
) = train_test_split(
    X_50_all_features, y_50_all_features, test_size=0.3, random_state=42
)

In [82]:
# Adicionando constante
X_train_50_all_features = sm.add_constant(X_train_50_all_features)
X_test_50_all_features = sm.add_constant(X_test_50_all_features)

In [83]:
# Criando o modelo e realizando predição
linear_model_50_all_features = sm.OLS(
    y_train_50_all_features, X_train_50_all_features
).fit()
pred_linear_model_50_all_features = linear_model_50_all_features.predict(
    X_test_50_all_features
)

In [84]:
# Cálculo do RMSE
rms = root_mean_squared_error(y_test_50_all_features, pred_linear_model_50_all_features)
print("RMSE", rms)

RMSE 0.7478685280228783


In [85]:
print(linear_model_50_all_features.summary(alpha=0.05))

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Tue, 10 Jun 2025   Prob (F-statistic):                nan
Time:                        09:27:22   Log-Likelihood:                 5853.4
No. Observations:                 186   AIC:                        -1.133e+04
Df Residuals:                       0   BIC:                        -1.073e+04
Df Model:                         185                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0449        inf          0        n

/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: divide by zero encountered in divide
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1795: RuntimeWarning: invalid value encountered in scalar multiply
  return 1 - (np.divide(self.nobs - self.k_constant, self.df_resid)
/Users/ciziks/Documentos/USP/Semestre 9/enem-embbedings-social-questions/.venv/lib/python3.11/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: divide by zero encountered in scalar divide
  return np.dot(wresid, wresid) / self.df_resid


### 6.3.4.B) Lasso


In [86]:
# Separando em conjunto de treino e teste
(
    X_train_50_lasso_all_features,
    X_test_50_lasso_all_features,
    y_train_50_lasso_all_features,
    y_test_50_lasso_all_features,
) = train_test_split(
    X_50_all_features, y_50_all_features, test_size=0.3, random_state=42
)

In [87]:
lasso_model_all_features_50 = LassoCV(
    alphas=[0.0001, 0.001, 0.01, 0.1, 1, 10], random_state=0
).fit(X_train_50_lasso_all_features, y_train_50_lasso_all_features)

pred_50_lasso_all_features = lasso_model_all_features_50.predict(
    X_test_50_lasso_all_features
)

print(
    "melhor r2 score:",
    lasso_model_all_features_50.score(
        X_train_50_lasso_all_features, y_train_50_lasso_all_features
    ),
)
print("melhor alpha:", lasso_model_all_features_50.alpha_)
print(
    "RMSE com o alpha escolhido:",
    root_mean_squared_error(y_test_50_lasso_all_features, pred_50_lasso_all_features),
)

melhor r2 score: 0.547574748756191
melhor alpha: 0.001
RMSE com o alpha escolhido: 0.48813975574107166


---
